In [1]:
from zero_churn_model.imports import * # basic imports
from zero_churn_model import helpers as hp
from tqdm import tqdm
from time import time 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

2024-09-05 06:46:11.426 | INFO     | zero_churn_model.config:<module>:11 - PROJ_ROOT path is: /home/zero/code-repos/zero_churn_model


# Dataset Description


[link here](https://www.openml.org/search?type=data&sort=runs&status=active&qualities.NumberOfInstances=between_10000_100000&qualities.NumberOfFeatures=between_100_1000&id=42758)

**Description**

This is the full version of the KDD Cup 2009 dataset

This Year's Challenge

Customer Relationship Management (CRM) is a key element of modern marketing strategies. The KDD Cup 2009 offers the opportunity to work on large marketing databases from the French Telecom company Orange to predict the propensity of customers to switch provider (churn), buy new products or services (appetency), or buy upgrades or add-ons proposed to them to make the sale more profitable (up-selling).

The most practical way, in a CRM system, to build knowledge on customer is to produce scores. A score (the output of a model) is an evaluation for all instances of a target variable to explain (i.e. churn, appetency or up-selling). Tools which produce scores allow to project, on a given population, quantifiable information. The score is computed using input variables which describe instances. Scores are then used by the information system (IS), for example, to personalize the customer relationship. An industrial customer analysis platform able to build prediction models with a very large number of input variables has been developed by Orange Labs. This platform implements several processing methods for instances and variables selection, prediction and indexation based on an efficient model combined with variable selection regularization and model averaging method. The main characteristic of this platform is its ability to scale on very large datasets with hundreds of thousands of instances and thousands of variables. The rapid and robust detection of the variables that have most contributed to the output prediction can be a key factor in a marketing application.

The challenge is to beat the in-house system developed by Orange Labs. It is an opportunity to prove that you can deal with a very large database, including heterogeneous noisy data (numerical and categorical variables), and unbalanced class distributions. Time efficiency is often a crucial point. Therefore part of the competition will be time-constrained to test the ability of the participants to deliver solutions quickly.

Task Description

The task is to estimate the churn, appetency and up-selling probability of customers, hence there are three target values to be predicted. The challenge is staged in phases to test the rapidity with which each team is able to produce results. A large number of variables (15,000) is made available for prediction. However, to engage participants having access to less computing power, a smaller version of the dataset with only 230 variables will be made available in the second part of the challenge.

Churn (wikipedia definition): Churn rate is also sometimes called attrition rate. It is one of two primary factors that determine the steady-state level of customers a business will support. In its broadest sense, churn rate is a measure of the number of individuals or items moving into or out of a collection over a specific period of time. The term is used in many contexts, but is most widely applied in business with respect to a contractual customer base. For instance, it is an important factor for any business with a subscriber-based service model, including mobile telephone networks and pay TV operators. The term is also used to refer to participant turnover in peer-to-peer networks.

Appetency: In our context, the appetency is the propensity to buy a service or a product.

Up-selling (wikipedia definition): Up-selling is a sales technique whereby a salesman attempts to have the customer purchase more expensive items, upgrades, or other add-ons in an attempt to make a more profitable sale. Up-selling usually involves marketing more profitable services or products, but up-selling can also be simply exposing the customer to other options he or she may not have considered previously. Up-selling can imply selling something additional, or selling something that is more profitable or otherwise preferable for the seller instead of the original sale.

The training set contains 50,000 examples. The first predictive 14,740 variables are numerical and the last 260 predictive variables are categorical. The last target variable is binary (-1,1).


# Load Base Data

In [2]:
fp = '../data/raw/kddcup09-orange-telcom-data.parquet'

# load base data and create a copy of it for 
df = pd.read_parquet(fp)

df.columns = [x.lower() for x in df.columns]
df = df[['churn'] + [x for x in df.columns if x != 'churn']]
print(df.shape)
df.sample(5)

(50000, 15001)


,churn,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var14991,var14992,var14993,var14994,var14995,var14996,var14997,var14998,var14999,var15000
38746,-1,0,0,0,0,0,0,0.0,0,0,...,?,?,Xrtjuqe,?,Q8_a,NBRvrWWx0Z,?,?,?,?
43011,-1,0,0,0,0,0,0,0.0,0,0,...,?,?,UDcUF98,?,Q8_a,NBRvrWWx0Z,?,?,?,?
39302,-1,0,0,0,0,0,0,0.0,0,0,...,?,?,wBBk8_4,?,KttQ,?,?,?,?,?
3409,-1,0,0,0,0,0,0,0.0,0,0,...,?,?,5i0Tzi7,?,Q8_a,NBRvrWWx0Z,?,vjl9npDZWXYGFIGZ,?,?
30060,-1,0,0,0,0,6,0,0.0,0,0,...,?,?,iMHYkhP,?,KttQ,NBRvrWWx0Z,?,?,?,?


# Drop Columns with One Unique Value

There are some columns which contain only the same value throughout the entire column:

In [3]:
# create a list of columns that have only one unique value
drop_cols = df.columns[df.nunique() == 1]
len(drop_cols)

1531

...since these columns don't contain any information, we can drop them:

In [4]:
# drop columns that have only one unique value 
df = df.drop(columns=drop_cols)

# Replace "?" with Null

Some columns have a mix of question marks and floats:

In [5]:
# show an example of a float column with ?
df.loc[975:985, 'var1176']

975    0.4
976    0.4
977    0.4
978    0.4
979    0.4
980      ?
981    0.4
982    0.4
983    0.4
984      ?
985    0.4
Name: var1176, dtype: object

We can treat these "?" values as null values:

In [6]:
# replace "?" values in the dataframe with np.nan
df = df.replace('?', np.nan)

# Set Data Types

Now, note that we have only `object` dtypes:

In [7]:
# print any columns that are not `object` dtypes (string)
dt = df.dtypes
dt[dt != 'object']

Series([], dtype: object)

We'll want to do some typecasting. According to the dataset documentation, "the first predictive 14,740 variables are numerical and the last 260 predictive variables are categorical", so we can use this to determine which columns are numerical:

In [8]:
colmap = dict([(int(x.replace('var', ''))+1, x) for x in df.columns if x != 'churn'])
int_cols = pd.Series(colmap)
int_cols

2            var1
3            var2
4            var3
5            var4
6            var5
           ...   
14996    var14995
14997    var14996
14998    var14997
14999    var14998
15000    var14999
Length: 13469, dtype: object

I have a function `sample_dataframe` which samples both rows and columns:

In [9]:
hp.display_source(hp.sample_dataframe)

```python
def sample_dataframe(df, nrows=1000, ncols=1000):
    """Create a smaller dataframe for initial testing"""
    # create a copy for temporary testing
    sl = df.T.sample(ncols).T.sample(nrows).copy()

    return sl

```

Columns with values greater than 14,740 should be string columns:

In [10]:
cols = int_cols[int_cols.index > 14740].tolist()
sl = hp.sample_dataframe(df[cols], 15, 5)
sl

,var14821,var14820,var14795,var14890,var14749
13211,NaN,NaN,7Gj8onV,5eCd,NaN
23863,ubXf6EK,NaN,XACjxqt,5eCd,NaN
42344,ubXf6EK,NaN,xJFdzwf,5eCd,NaN
12593,ubXf6EK,NaN,XAC65c7,5eCd,NaN
26557,NaN,NaN,RXURMDY,zSxe,kLIf
44998,ubXf6EK,NaN,xJFjktZ,5eCd,NaN
21409,NaN,NaN,Udrwr9F,zSxe,NaN
34936,NaN,NaN,Udrwr9F,5eCd,NaN
17550,ubXf6EK,NaN,Gwc_fIH,5eCd,NaN
48722,ubXf6EK,NaN,7Gj8onV,5eCd,NaN


...and the rest of the columns should be numerical:

In [11]:
numerical_cols = int_cols[int_cols.index <= 14740].tolist()
sl = hp.sample_dataframe(df[numerical_cols], 15, 5)
sl

,var5150,var13069,var4768,var7537,var4841
357,0,9,0,0,0
11896,0,0,0,0,0
16706,0,0,0,0,0
26813,0,0,0,0,0
32919,0,0,0,0,0
12415,0,0,0,0,0
38569,0,0,0,0,0
29636,0,0,0,0,0
12647,0,0,0,0,0
1415,0,0,0,0,0


We can't inspect all of them, but it appears that these columns are numeric. Let's try to cast the other columns to `float`, but first we'll do it on a sample dataframe before trying on the very large dataframe:

In [12]:
sl = hp.sample_dataframe(df[numerical_cols], 1000, 1000)

In [13]:
for col in [x for x in numerical_cols if x in sl.columns]:
    sl[col] = sl[col].astype(float)

...that worked, so we'll do it on `df` now:

In [ ]:
for col in numerical_cols + ['churn']:
    try:
        df[col] = df[col].astype(float)
    except Exception as e:
        print(f"column='{col}'")
        raise e

In [ ]:
df.dtypes.value_counts()

# Set Binary Columns

If there are only two unique values, we'll convert this to a 0,1 bool column:

In [ ]:
bool_cols = df.columns[df.nunique() == 2]

In [ ]:
for col in bool_cols:
    df[col] = pd.factorize(df[col])[0]

# Interpolate Null Values

Now, we still have null values in columns that we need to deal with. Let's start with the numerical columns that aren't binary:

In [ ]:
numerical_nonbool_cols = [x for x in numerical_cols if x not in bool_cols]
len(numerical_nonbool_cols)

...recalling that we know that `numerical_cols` are specifically numerical since we cast them that way. 

We'll do a simple mean interpolation for now:

In [ ]:
%%time
for col in numerical_nonbool_cols:
    df[col] = df[col].fillna(df[col].mean())

In [ ]:
# we don't have any null values in numerical cols 
assert df[numerical_cols].isnull().sum().sum() == 0

We still have some categorical columns to deal with 

In [ ]:
string_nonbool_cols = [x for x in df.columns if x not in numerical_cols and x not in bool_cols]
len(string_nonbool_cols)

In [ ]:
# display number of null values
df[string_nonbool_cols].isnull().sum().sum()

In [ ]:
for col in string_nonbool_cols:
    df[col] = pd.factorize(df[col])[0]

In [ ]:
# display number of null values
assert df[string_nonbool_cols].isnull().sum().sum() == 0

Now, we notice that some of the categorical columns have a very large number of categorical values:

In [ ]:
x = df['var14868'].value_counts()
len(x)

this won't really work very well, so let's use frequency encoding for these columns

In [ ]:
high_cardn_cols = []
for col in string_nonbool_cols:
    vc = df[col].value_counts()
    if len(vc) > 10:
        high_cardn_cols.append(col)
len(high_cardn_cols)

In [ ]:
for col in high_cardn_cols:
    vc = df[col].value_counts()
    vc = vc / vc.sum()
    df[col] = df[col].map(vc)

In [ ]:
sl = hp.sample_dataframe(df[high_cardn_cols], 15, 5)
sl

In [ ]:
# list any string columns
x = df.dtypes
x[x == 'object']

In [ ]:
# check that there aren't any more missing values
assert not df.isnull().sum().sum()

# Archive Dataframe

Now that there aren't any missing values and we've set our data types, we can archive the dataframe.

In [ ]:
df.to_parquet('../data/interim/interim-data.parquet')